This notebook contains some tests to decide on an experimental design for the simulatability experiment.

In [1]:
import krippendorff


In [2]:
from scipy.stats import ttest_ind

In [3]:

import numpy as np 
import scipy.stats as stats 
import matplotlib.pyplot as plt


In [4]:
import pandas as pd
import numpy as np

In [5]:
import random

## Hase et al. 2020.

This is a within-subject* design with 4 phases: (1) Predictions only, (2) Pre-learn test, (3) Teaching: Predictions + Explanations, (4) Eval.

Phase 1 and 3 share a set of documents as do 2 and 4.

Result: Report **average change** in user accuracy per explanation method (phase 2 vs. 4), CI and p values of mean

Additional parameters in Hase et al.:
- Balance data "by model correctness" so random guessing can't succeed: *"we ensure that true positives, false positives,
true negatives, and false negatives are equally represented in the inputs. [...] We confirm user understanding of the data
balancing in our screening test"*
- Forced choice, to not "favor overly niche explanations" (like in Ribeiro et al.)
- Separate teach and test phases
- Pre prediction phase to obtain a baseline
- **All users see the same examples**


*: One explanation method per user, some users repeat the experiment with a new dataset!?!



### Feasability

In [6]:
columns_experiment = ["user_id", "document_id", "user_label"]


In [7]:
def guess(detector_label,p):
    return detector_label if bool(np.random.choice([0,1],p=[1-p, p])) else not detector_label

In [8]:
import sqlite3
import pandas as pd
# Create your connection.
connection = sqlite3.connect("../survey/db.db")

user_df = pd.read_sql_query("SELECT * FROM users", connection)

In [9]:

n_learn = 16
n_eval = 16
n_users = 10

mu_got_it_right_pre=0.5
sigma_got_it_right_pre=0.05
mu_gain = 0.2
sigma_gain = 0.1

In [10]:
import requests
import json

In [11]:
url = "http://localhost:3002"

In [12]:
documentNr = 1
label = 0

In [13]:
user_df

,ID,access_token,current_phase,detector,explainer,document_order_a,document_order_b
0,1,SNNCXN,1,DetectorDetectGPT,SHAP_Explainer,"[12,11,1,6,16,15,2,9,13,8,0,4,14,7,17,5,3,10]","[15,14,0,10,9,7,13,8,3,5,16,2,4,12,1,6,17,11]"
1,2,SCTLCS,1,DetectorGuo,SHAP_Explainer,"[8,16,6,4,9,7,17,1,12,0,2,13,11,15,10,5,3,14]","[9,15,6,3,13,14,16,12,2,17,4,1,7,8,10,0,5,11]"
2,3,QGDAAI,1,DetectorRadford,SHAP_Explainer,"[6,4,11,17,10,1,15,14,13,5,0,16,2,12,7,8,3,9]","[11,15,2,7,4,14,3,17,6,9,1,8,16,0,5,10,12,13]"
3,4,HKHDUC,1,DetectorDetectGPT,LIME_Explainer,"[8,0,1,13,14,5,11,6,16,12,10,15,4,9,3,7,2,17]","[14,11,7,13,2,10,4,17,6,1,9,3,15,5,0,8,12,16]"
4,5,IDWRGD,1,DetectorGuo,LIME_Explainer,"[6,4,8,9,5,14,0,13,15,12,10,16,7,1,3,2,17,11]","[10,15,17,1,0,11,9,2,13,7,16,14,3,4,5,12,6,8]"
5,6,MKMXSA,1,DetectorRadford,LIME_Explainer,"[5,15,3,11,1,0,6,2,10,12,16,17,14,4,13,9,8,7]","[2,9,16,15,1,6,3,17,7,11,0,13,4,8,10,5,12,14]"
6,7,NLCFZC,1,DetectorDetectGPT,Anchor_Explainer,"[5,8,9,0,15,2,14,17,16,1,4,12,7,11,3,6,13,10]","[0,4,2,7,6,3,13,10,1,5,8,15,12,11,9,17,16,14]"
7,8,NMGLSY,1,DetectorGuo,Anchor_Explainer,"[3,5,17,10,13,2,6,1,16,14,15,12,4,11,0,8,9,7]","[14,2,15,13,1,12,17,4,0,9,3,7,6,8,11,10,16,5]"
8,9,TRDHAH,1,DetectorRadford,Anchor_Explainer,"[14,4,9,0,17,7,11,10,13,15,1,2,6,5,16,3,8,12]","[11,17,4,6,0,5,12,13,1,2,16,8,7,14,9,15,3,10]"
9,10,YZLEER,1,DetectorDetectGPT,SHAP_Explainer,"[2,1,13,8,6,12,15,14,7,4,0,17,3,11,9,10,16,5]","[12,14,15,4,6,13,3,8,2,11,16,9,1,0,5,10,7,17]"


In [14]:
df_user_study = pd.read_pickle("./dataset_user_study_new.pkl")

In [15]:
users = []
user_dist_without = lambda : np.clip(np.random.normal(mu_got_it_right_pre, sigma_got_it_right_pre, 1)[0], 0,1)
user_dist_gain = lambda : np.clip(np.random.normal(mu_gain, sigma_gain, 1)[0], -1,1)
for idx, user in user_df.iterrows():
    res = requests.get(url+"/auth/"+ user["access_token"])
    auth_token = json.loads(res.text)
    headers = {'Content-Type': 'application/json','Authorization': "Bearer "+auth_token, "Content-Type": "application/json",}

    requests.post(url+"/submitParticipantInfo", json={
    "has_seen_explanation_methods_before": "yes",
    "has_seen_OTHERS_before": "yes",
    "level_of_expertise": "is-researcher-explainability",
    "familiarity_with_chatgpt": "occasional-use",
    "prefers_monochromatic_methods": "yes" if user["access_token"] == "DDEBUG" else "no"
    }, headers=headers)
    # go to phase 2
    requests.post(url+"/completeCurrentPhase", json={"expected": 0}, headers=headers)
    requests.post(url+"/completeCurrentPhase", json={"expected": 1}, headers=headers)
    requests.post(url+"/completeCurrentPhase", json={"expected": 2}, headers=headers)

    user_df = pd.read_sql_query("SELECT * FROM users", connection) # update as group is assigned now
    user = user_df.iloc[idx]
   # print(user[["detector", "explainer"]])
    df_user_documents = df_user_study.loc[df_user_study.groupby("Detector").groups[user["detector"]],:].reset_index(drop=True)
    for doc_nr, row in df_user_documents.iterrows():
        p_without = user_dist_without()
        requests.post(url+"/submitPhase2", json={"ID": doc_nr, "label": guess(row["f(b)"], p_without)}, headers=headers)
    requests.post(url+"/completeCurrentPhase", json={"expected": 3}, headers=headers)
    requests.post(url+"/completeCurrentPhase", json={"expected": 4}, headers=headers)
    for doc_nr, row in df_user_documents.iterrows():
        p_with = np.clip(p_without + user_dist_gain(), 0,1)
        requests.post(url+"/submitPhase4", json={"ID": doc_nr, "label": guess(row["f(b)"], p_with)}, headers=headers)
    requests.post(url+"/completeCurrentPhase", json={"expected": 5}, headers=headers)

In [16]:
user_df.groupby(["detector", "explainer"])["ID"].count()

detector           explainer       
DetectorDetectGPT  Anchor_Explainer    4
                   LIME_Explainer      4
                   SHAP_Explainer      5
DetectorGuo        Anchor_Explainer    4
                   LIME_Explainer      4
                   SHAP_Explainer      4
DetectorRadford    Anchor_Explainer    4
                   LIME_Explainer      4
                   SHAP_Explainer      4
Name: ID, dtype: int64

In [17]:
def user_metrics(df_user_responses, df_user_study):
    detector = df_user_responses.iloc[0]["detector"]
    # explainer = df_user_responses.iloc[0]["explainer"]
    df_user_documents = df_user_study.loc[df_user_study.groupby("Detector").groups[detector],:].reset_index(drop=True)
    detector_predictions = df_user_documents["f(b)"].astype(bool)

    user_responses = df_user_responses.loc[df_user_responses.groupby("document_nr")["timestamp"].idxmax()].set_index("document_nr")["label"].astype(bool) # only keep most recent response
    # display(user_responses)
    # display(detector_predictions)
    TP = ((detector_predictions) & (user_responses)).sum()
    FP = ((~detector_predictions) & (user_responses)).sum()

    TN = ((~detector_predictions) & (~user_responses)).sum()
    FN = ((detector_predictions) & (~user_responses)).sum()

    acc = (TP+TN) / (TP+FP+TN+FN)
    # print("acc", acc)
    # print("TP", TP)
    # print("FP", FP)
    # print("TN", TN)
    # print("FN", FN)


    assert sum([TP,FP,TN,FN]) == len(detector_predictions), "Check that input is bool"
    assert (acc ==(user_responses == detector_predictions).sum() / len(detector_predictions)), "Check that input is bool: acc"

    return pd.DataFrame([(TP,TN,FP,FN, acc)], columns=["TP","TN","FP","FN", "User Accuracy"])

In [18]:
u = user_df.set_index("ID").rename_axis("user_id")[["explainer", "detector"]]


In [19]:
df_phase_2 = pd.read_sql_query("SELECT responses_phase_2.*, users.detector, users.explainer FROM responses_phase_2 INNER JOIN users ON responses_phase_2.user_id = users.ID", connection)
df_phase_4 = pd.read_sql_query("SELECT responses_phase_4.*, users.detector, users.explainer FROM responses_phase_4 INNER JOIN users ON responses_phase_4.user_id = users.ID", connection)

metrics_phase_4 = df_phase_4.groupby(["user_id"]).apply(lambda df_user_responses : user_metrics(df_user_responses,df_user_study))
metrics_phase_2 = df_phase_2.groupby(["user_id"]).apply(lambda df_user_responses : user_metrics(df_user_responses,df_user_study))


difference = metrics_phase_4 - metrics_phase_2 


In [21]:
difference.join(u).groupby(["detector"])["User Accuracy"].mean()

detector
DetectorDetectGPT    0.217949
DetectorGuo          0.162037
DetectorRadford      0.231481
Name: User Accuracy, dtype: float64

In [22]:
difference.join(u).groupby(["explainer"])["User Accuracy"].mean()

explainer
Anchor_Explainer    0.166667
LIME_Explainer      0.203704
SHAP_Explainer      0.239316
Name: User Accuracy, dtype: float64

In [23]:
difference.join(u).groupby(["detector", "explainer"])["User Accuracy"].mean()

detector           explainer       
DetectorDetectGPT  Anchor_Explainer    0.180556
                   LIME_Explainer      0.152778
                   SHAP_Explainer      0.300000
DetectorGuo        Anchor_Explainer    0.138889
                   LIME_Explainer      0.194444
                   SHAP_Explainer      0.152778
DetectorRadford    Anchor_Explainer    0.180556
                   LIME_Explainer      0.263889
                   SHAP_Explainer      0.250000
Name: User Accuracy, dtype: float64

In [24]:
metrics_phase_4.join(u).index.get_level_values(0)

Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37],
      dtype='int64', name='user_id')

In [25]:
def highlight_significant(row, props=''):
  #  display(s)
    styles = [''] * len(row)
    styles[2] = 'font-weight: bold' if row["p value"] <= 0.05 else ''
    return styles

In [26]:
latex_output = []

In [27]:
def get_aggregate_results(groupby, label, caption):
    tvalues = []
    pvalues = []
    for name, group_2 in metrics_phase_2.join(u).groupby(groupby):
        group_4 =  metrics_phase_4.join(u)[np.all(metrics_phase_4.join(u)[groupby].values == name, axis=1)]
        tvalue, pvalue = ttest_ind(group_2["User Accuracy"],group_4["User Accuracy"])
        tvalues.append(tvalue)
        pvalues.append(pvalue)

    df_aggregate_results = pd.DataFrame(difference.join(u).groupby(groupby)["User Accuracy"].mean())

    df_aggregate_results = df_aggregate_results.join(pd.DataFrame(metrics_phase_2.join(u).groupby(groupby)["User Accuracy"].mean()).rename({"User Accuracy":"Before"}, axis=1))
    df_aggregate_results = df_aggregate_results.join(pd.DataFrame(metrics_phase_4.join(u).groupby(groupby)["User Accuracy"].mean()).rename({"User Accuracy":"After"}, axis=1))

    df_aggregate_results["t value"] = tvalues
    df_aggregate_results["p value"] = pvalues

    df_aggregate_results.rename(columns={"User Accuracy":"Increase in User Accuracy"}, inplace=True)
    df_aggregate_results = df_aggregate_results.reindex(sorted(df_aggregate_results.columns), axis=1)
    result = df_aggregate_results.style.apply(highlight_significant, axis=1)\
        .map_index(lambda v: "rotatebox:{45}--rwrap;", level=0, axis=1).format(precision=2).hide(["t value"], axis=1).format_index(escape="latex", axis=0)
    latex_output.append(result.to_latex(environment="longtable", 
                                        convert_css=True, 
                                        clines="all;data", 
                                        hrules=True, 
                                        caption=caption, 
                                        label=label))
    return result

In [28]:
get_aggregate_results(["detector"], "resultsuserstudydetector", "Results aggregated by detector")

,After,Before,Increase in User Accuracy,p value
detector,,,,
DetectorDetectGPT,0.73,0.51,0.22,0.00
DetectorGuo,0.67,0.50,0.16,0.00
DetectorRadford,0.73,0.50,0.23,0.00


In [29]:
get_aggregate_results(["explainer"], "resultsuserstudyexplainer", "Results aggregated by explainer")

,After,Before,Increase in User Accuracy,p value
explainer,,,,
Anchor\_Explainer,0.69,0.53,0.17,0.01
LIME\_Explainer,0.71,0.51,0.20,0.00
SHAP\_Explainer,0.71,0.47,0.24,0.00


In [30]:
get_aggregate_results(["explainer", "detector"], "resultsuserstudyexplainerdetector", "Results aggregated by explainer and detector pairing")

In [31]:
for  l in latex_output:
    print(l)

\begin{longtable}{lrrrr}
\caption{Results aggregated by detector} \label{resultsuserstudydetector} \\
\toprule
 & \rotatebox{45}{After} & \rotatebox{45}{Before} & \rotatebox{45}{Increase in User Accuracy} & \rotatebox{45}{p value} \\
detector &  &  &  &  \\
\midrule
\endfirsthead
\caption[]{Results aggregated by detector} \\
\toprule
 & \rotatebox{45}{After} & \rotatebox{45}{Before} & \rotatebox{45}{Increase in User Accuracy} & \rotatebox{45}{p value} \\
detector &  &  &  &  \\
\midrule
\endhead
\midrule
\multicolumn{5}{r}{Continued on next page} \\
\midrule
\endfoot
\bottomrule
\endlastfoot
DetectorDetectGPT & 0.73 & 0.51 & \bfseries 0.22 & 0.00 \\
\cline{1-5}
DetectorGuo & 0.67 & 0.50 & \bfseries 0.16 & 0.00 \\
\cline{1-5}
DetectorRadford & 0.73 & 0.50 & \bfseries 0.23 & 0.00 \\
\cline{1-5}
\end{longtable}

\begin{longtable}{lrrrr}
\caption{Results aggregated by explainer} \label{resultsuserstudyexplainer} \\
\toprule
 & \rotatebox{45}{After} & \rotatebox{45}{Before} & \rotatebox{45}

In [32]:




# # "Reproducing" information from Table 1/2 in Hase et al.:


# user_metrics_eval = df_phase_4.groupby(["user_id"]).apply(lambda x : user_metrics(x,df_user_study)) # TODO
# df_change = user_metrics_eval - user_metrics_pre
# df_change = df_change.rename(columns={"User Accuracy": "Change in User Accuracy"})

# user_acc_col = df_change["Change in User Accuracy"] # for convenience

# # use student t for low number of samples
# lower, upper = stats.t.interval(
# confidence=0.95, 
# df=len(user_acc_col)-1, # degrees of freedom = # samples - 1 for mean
#             loc=user_acc_col.mean(), 
#             scale=stats.sem(user_acc_col)
#             ) 

# p_val = ttest_ind(user_metrics_eval["User Accuracy"],user_metrics_pre["User Accuracy"]).pvalue

# k_alpha = krippendorff.alpha(reliability_data=df_phase_4.groupby(["user_id"]).apply(lambda df : user_responses.astype(int).to_list()).to_list())



# lower_b, upper_b = stats.bootstrap((user_acc_col,), np.mean, confidence_level=0.95,).confidence_interval

# # print results
# #  print("Mean change in acc",user_acc_col.mean())
# # print("CI for mean change: [{},{}]".format(lower,upper))

# ##    print("CI by bootstrap: [{},{}]".format(lower_b, upper_b))


# #  print("p=%.10f" % p_val, "significant (< 0.05)" if p_val < 0.05 else "NOT significant (> 0.05)")
# #   print("Krippendorff between users: {}".format(k_alpha))
# return p_val

In [33]:
# def evaluate_user_study(df_user_study, df_phase_2, df_phase_4):
#     # "Reproducing" information from Table 1/2 in Hase et al.:
#     user_metrics_pre = df_phase_2.groupby(["user_id"]).apply(lambda x : user_metrics(x,df_user_study))

#     user_metrics_eval = df_phase_4.groupby(["user_id"]).apply(lambda x : user_metrics(x,df_user_study)) # TODO
#     df_change = user_metrics_eval - user_metrics_pre
#     df_change = df_change.rename(columns={"User Accuracy": "Change in User Accuracy"})

#     user_acc_col = df_change["Change in User Accuracy"] # for convenience

#     # use student t for low number of samples
#     lower, upper = stats.t.interval(
#     confidence=0.95, 
#     df=len(user_acc_col)-1, # degrees of freedom = # samples - 1 for mean
#               loc=user_acc_col.mean(), 
#               scale=stats.sem(user_acc_col)
#               ) 
    
#     p_val = ttest_ind(user_metrics_eval["User Accuracy"],user_metrics_pre["User Accuracy"]).pvalue

#     k_alpha = krippendorff.alpha(reliability_data=df_phase_4.groupby(["user_id"]).apply(lambda df : user_responses.astype(int).to_list()).to_list())



#     lower_b, upper_b = stats.bootstrap((user_acc_col,), np.mean, confidence_level=0.95,).confidence_interval
    
#     # print results
#   #  print("Mean change in acc",user_acc_col.mean())
#    # print("CI for mean change: [{},{}]".format(lower,upper))

# ##    print("CI by bootstrap: [{},{}]".format(lower_b, upper_b))

    
#   #  print("p=%.10f" % p_val, "significant (< 0.05)" if p_val < 0.05 else "NOT significant (> 0.05)")
#  #   print("Krippendorff between users: {}".format(k_alpha))
#     return p_val
    

In [34]:
# def simulate_hase(

#         n_learn = 16,
#         n_eval = 16,
#         n_users = 10,

#         mu_got_it_right_pre=0.5,
#         sigma_got_it_right_pre=0.05,

#         mu_gain = 0.1,
#         sigma_gain = 0.1,


# ):
#     users = []
#     user_dist_without = lambda : np.clip(np.random.normal(mu_got_it_right_pre, sigma_got_it_right_pre, 1)[0], 0,1)
#     user_dist_gain = lambda : np.clip(np.random.normal(mu_gain, sigma_gain, 1)[0], -1,1)
#     for i in range(1, n_users+1):
#         p_without = user_dist_without()
#         p_with = np.clip(p_without + user_dist_gain(), 0,1)
#         users.append(("u_%s" % i, p_without ,p_with))
#     documents_learn_1_2 = ["l_%s" % i for i in range(1,n_learn+1)]
#     documents_pre_eval = ["e_%s" % i for i in range(1,n_eval+1)]

#     df_detector_output = mock_detector_responses(documents_pre_eval)

#     responses_pre, responses_eval = mock_user_responses(df_detector_output, documents_pre_eval, users)
#     df_pre =pd.DataFrame(responses_pre, columns=columns_experiment)
#     df_eval =pd.DataFrame(responses_eval, columns=columns_experiment)

#  #   print("# responses pre", len(responses_pre))
#   #  print("# responses pre per method", len(responses_pre)/3)
#    # print("Each user saw {} instances. ".format(2*n_learn + 2*n_eval) )
#    # print("Used {} unique documents. A set of {} in phase 1 and 3; and a set of {} in phase 2 and 4.".format(n_learn + n_eval,n_learn, n_eval))

# #    print("Results based on {} unique eval documents.".format(n_eval))

# #    print("Results based on {} datapoints.".format(len(responses_eval)))
#     p_value = evaluate_user_study(df_detector_output, df_pre, df_eval)
#     return p_value

: 